# WIP: Science Notebook "Exploration" of GW sources

The story: Look at the exciting previous gravitational wave event GW170817 in NGC 4993. This was the GW event where two NS coalesced and therefore it is also interesting to study the multiwavelength from the event. 

Steps: 
1. Use the NED tool (https://ned.ipac.caltech.edu/uri/NED::GWFevents/) to find potential host galaxies (note NGC 4493 is on this list... need to narrow down using other data -- what? how?). 

2. Find what other X-ray data exist on the different sources. 

3. Make thumbnail images of potential host galaxies. 

4. Multi-wavlelength lightcurves of decay. (Would be really cool to make the type of plot of the different spectra as function of time. see https://www.eso.org/public/images/eso1733j/ ) 

.... Other things? ...

## First, Set-up: Import the Python modules we'll be using.

In [54]:
# Suppress unimportant warnings.
import warnings
warnings.filterwarnings("ignore", module="astropy.io.votable.*")
warnings.filterwarnings("ignore", module="pyvo.utils.xml.*")
warnings.filterwarnings('ignore', '.*RADECSYS=*', append=True)

import matplotlib.pyplot as plt
import numpy as np

# For downloading files
from astropy.utils.data import download_file

from astropy.coordinates import SkyCoord
from astropy.io import fits
from astropy.nddata import Cutout2D
import astropy.visualization as vis
from astropy.wcs import WCS
from astroquery.ned import Ned

import pyvo as vo

### NEW
import requests
from astropy.io import ascii
from astropy.table import Table
import csv

In [32]:
%matplotlib inline

## Now, we can begin the science exploration:

### Step 1: Use the NED tool (https://ned.ipac.caltech.edu/uri/NED::GWFevents/) to find potential host galaxies

According to the NED API page, to grab the top 20 list of galaxies in the localization of event, 
sorted by 2MASS absolute K-band magnitude as a CSV output, you need the following URL
(see https://ned.ipac.caltech.edu/Documents/Guides/Interface/GWF):
https://ned.ipac.caltech.edu/uri/NED::GWFglist/csv/GW170817/latest/20

In [160]:
url = "https://ned.ipac.caltech.edu/uri/NED::GWFglist/csv/GW170817/latest/20"
response = requests.get(url, allow_redirects=True)

data = response.content.decode('utf-8')

Let's look at this information. The correct host galaxy is NGC4993 (idx=4). 

In [162]:
# Look at CSV content from url file 
csvdata = list(csv.reader(data.splitlines(), delimiter=','))
t=Table(rows=csvdata[1:], names=csvdata[0])
t.show_in_notebook()

idx,objname,ra,dec,pretype,z,zunc,DistMpc,DistMod,DistMethod,m_Ks,k_msig_k20fe,M_Ks,Merr_Ks,m_FUV,magerr_fuv,M_FUV,Merr_FUV,dP_dV,dP_dA
0,NGC 5078,199.95825,-27.41038888889,G,0.00723200012,1.99999995e-05,30.113245010375977,32.3937873840332,Kinematic,7.151000022888184,0.013000000268220901,-25.242786407470703,0.013000000268220901,nan,nan,nan,nan,0.0016472022029977132,58.94887964418315
1,NGC 5061,199.521125,-26.837222222220007,G,0.00694500003,6.29999995e-05,24.700000762939453,31.963485717773438,Median,7.366000175476074,0.014000000432133675,-24.597484588623047,0.014000000432133675,17.97249984741211,0.09267310053110123,-13.990985870361328,0.09267310053110123,0.0009884641085415855,98.66444883107857
2,NGC 4970,196.89058333333,-24.00855555556,G,0.0108580003,0.000117000003,42.70000076293945,33.152137756347656,Median,9.170000076293945,0.019999999552965164,-23.98213768005371,0.019999999552965164,nan,nan,nan,nan,0.0014576997107457305,67.43992470295204
3,IC 4197,197.01804166667,-23.79686111111,G,0.0100969998,1.99999995e-05,45.85000228881836,33.306697845458984,Median,9.352999687194824,0.027000000700354576,-23.953697204589844,0.027000000700354576,nan,nan,nan,nan,0.0018727781291845522,136.56825676220575
4,NGC 4993,197.44875,-23.38388888889,G,0.00972700026,4.99999987e-05,41.20000076293945,33.074485778808594,Median,9.329999923706055,0.026000000536441803,-23.74448585510254,0.026000000536441803,19.955799102783203,99.0,-13.11868667602539,99.0,0.0099267137099688,294.12262609641215
5,IC 4180,196.735375,-23.91711111111,G,0.00991399959,5.99999985e-05,41.280792236328125,33.078739166259766,Kinematic,9.694000244140625,0.027000000700354576,-23.38473892211914,0.027000000700354576,19.017000198364258,0.11121299862861633,-14.061738967895508,0.11121299862861633,0.0014636385238314357,56.07310922978644
6,IC 0874,199.75216666667,-27.628555555560002,G,0.0077490001,3.30000003e-05,31.25,32.47425079345703,Median,9.769000053405762,0.028999999165534973,-22.705249786376953,0.028999999165534973,20.5664005279541,99.0,-11.90785026550293,99.0,0.0024446315427621313,89.51009000297057
7,ESO 508- G 033,199.096875,-26.56152777778,G,0.011094,0.00012,46.194183349609375,33.32293701171875,Kinematic,10.876999855041504,0.04800000041723251,-22.445938110351562,0.04800000041723251,17.489999771118164,0.047068800777196884,-15.832937240600586,0.047068800777196884,0.0028013262414659577,153.68893968985364
8,ESO 508- G 010,196.90716666667,-23.578944444439998,G,0.0104740001,0.00013,43.612571716308594,33.19805908203125,Kinematic,11.303000450134277,0.08799999952316284,-21.895057678222656,0.08799999952316284,nan,nan,nan,nan,0.0034574610436782843,164.10119405342184
9,IC 0879,199.91904166667,-27.42897222222,G,0.00796500035,3.9999999e-05,33.165374755859375,32.603424072265625,Kinematic,11.079999923706055,0.0729999989271164,-21.52342414855957,0.0729999989271164,17.311899185180664,0.03985830023884773,-15.291524887084961,0.03985830023884773,0.0020721203855033044,58.94887964418315


Good news is that the host galaxy is in the list. But we don't know this a priori and we also likely don't have real time monitoring of these potential candidates. However, in the case of GW170817, optical monitoring did discover an EM counterpart in NGC 4993. So we can now look at this galaxy in more detail. 

POTENTIAL IDEA: Can we use GCN data? Is that useful for our science case and to exemplify other NAVO services?